# Lecture: Monodromy

In [1]:
using HomotopyContinuation

<hr style="border:1px solid gray"> </hr>
For more information about monodromy and computing  monodromy groups using numerical algebraic geometry see 

[[*Numerical computation of Galois groups*](https://arxiv.org/pdf/1605.07806.pdf)] Hauenstein, Rodriguez, and Sottile. Foundations of Computational Mathematics, 18 (2018)

For more information about solve polynomial systems using monodromy see

[[*Solving polynomial systems via homotopy continuation and monodromy*](https://arxiv.org/pdf/1609.08722.pdf)]  Duff,  Hill,  Jensen,  Lee,  Leykin, and  Sommars.
IMA Journal of Num. Anal. (2018)

<hr style="border:1px solid gray"> </hr>

<img src="PictureMonodromy1.png" width="500" style="float:right">

## Branched Covers

As in Sascha's talk, we consider a family $F(x;p)$ of $0$-dimensional polynomial systems parametrized by $Q=\mathbb{C}^k$
$$Z = \{(x,p) \in \mathbb{C}^n \times Q \mid F(x;p) =0\} \subset \mathbb{C}^n \times Q$$
$$\,\,\,\, \downarrow \pi $$
$$Q$$

Over *most* parameters, there are $d$ solutions.

The map $\pi$ is called a $d$-to-$1$ **branched cover** 

The **discriminant** (or branch locus) $D \subsetneq Q$ are the parameter values for which the fibres do not consist of $d$ distinct points.

The complement of the discriminant $U = Q \backslash D$ is called the set of the regular values.

The map $\pi:Z|_{\pi^{-1}(U)} \to U$ is a $d$-to-$1$ **covering space**.







### Example

$$F(x;a,b) = x^2+ax+b$$


In [58]:
@var x a b
F = System([x^2 + a * x + b], parameters = [a, b])

System of length 1
 1 variables: x
 2 parameters: a, b

 b + a*x + x^2

<p style="border:3px; border-style:solid; padding: 0.5em; text-align:center">
Monodromy: a loop $\gamma$ in $U$ based at $p$ induces a permutation $g_\gamma$ of the fibre $\pi^{-1}(p)$.
</p>


Let's move in a loop

<img src="PictureMonodromy2.png" width="500">


$$\overbrace{\underbrace{(x^2-1)}_{p_0=[0,-1]} \to \underbrace{(x^2-i)}_{p_1=[0,-i]} \to \underbrace{(x^2+1)}_{p_2=[0,1]} \to \underbrace{(x^2+i)}_{p_3=[0,i]} \to \underbrace{(x^2-1)}_{p_0=[0,-1]}}^{\gamma}$$

In [59]:
S₀ = [[-1],[1]] #the solutions to the system at p_0
p₀ = [0,-1]
p₁ = [0, im]
p₂ = [0,1]
p₃ = [0,-im]

2-element Array{Complex{Int64},1}:
 0 + 0im
 0 - 1im

In [60]:
#p0->p1
S₁ = solve(F,S₀; start_parameters=p₀,target_parameters=p₁)
#p1->p2
S₂ = solve(F,S₁; start_parameters=p₁,target_parameters=p₂)
#p2->p3
S₃ = solve(F,S₂; start_parameters=p₂,target_parameters=p₃)
#p3->p0 again
S₀2 = solve(F,S₃; start_parameters=p₃,target_parameters=p₀)

Result with 2 solutions
• 2 paths tracked
• 2 non-singular solutions (2 real)
• random_seed: 0x111d1b8e


In [61]:
show(stdout,"text/plain",hcat(S₀,solutions(S₁),solutions(S₂),solutions(S₃),solutions(S₀2)))

2×5 Array{Array{Complex{Float64},1},2}:
 [-1.0+0.0im]  [-0.707107+0.707107im]  [0.0+1.0im]  [0.707107+0.707107im]   [1.0+0.0im]
 [1.0+0.0im]   [0.707107-0.707107im]   [0.0-1.0im]  [-0.707107-0.707107im]  [-1.0+0.0im]

In [6]:
#This function automates the process above, given a parametrized polynomial system F, start solutions 
# Sols, and a loop γ which is just a sequence of parameter values starting and ending with the parameter
# value for Sols

function monodromy_loop(F,Sols,γ)
    S=Sols;
    P=γ[1]
    for NewP in γ[2:end]
        newS=solve(F,S; start_parameters=P, target_parameters=NewP)
        S=newS
        P=NewP
    end
    return S
end

monodromy_loop (generic function with 1 method)

In [7]:
#Here's the loop we did earlier, using the new function
M=monodromy_loop(F,S₀,[p₀,p₁,p₂,p₃,p₀]);
solutions(M)

2-element Array{Array{Complex{Float64},1},1}:
 [1.0 + 0.0im]
 [-1.0 + 0.0im]

# Solving systems using monodromy - a new way to solve

What if we only knew about the solution $s_1=1$ of $x^2-1=0$?

Then performing the monodromy loop *discovers* the solution $s_2=-1$

In [62]:
solutions(monodromy_loop(F,[1],[p₀,p₁,p₂,p₃,p₀]))

1-element Array{Array{Complex{Float64},1},1}:
 [-1.0 + 0.0im]

What if we didn't know any solutions to a polynomial system? 

In [63]:
@var a[1:4]
@var b[1:5]
@var x,y
Sparse = System([a[1]+a[2]*x^2+a[3]*y^2+a[4]*x^2*y^2,
                 b[1]+b[2]*x^2+b[3]*x^2*y^2+b[4]*x^2*y^4+b[5]*x^4*y^2],
                 parameters=vcat(vec(a),vec(b)))
MySparseParameters=randn(Float64,9)
MySparseSystem=System(evaluate(Sparse.expressions,Sparse.parameters=>MySparseParameters))

System of length 2
 2 variables: x, y

 -0.506056017548317 + 0.618632832982324*x^2*y^2 + 0.361857264121154*x^2 - 1.57181367445727*y^2
 0.903438428435694 - 0.9960887061833*x^2*y^2 + 1.01891230437757*x^2*y^4 + 1.09058934838748*x^4*y^2 - 1.13165678982091*x^2

This system lives in the set of all polynomial systems supported on 

<img src="PictureMonodromy9.png" width="500">

Rather than solving this using a Bezout, or Polyhedral start system, we will try monodromy.

But we don't know a start solution!

<img src="PictureMonodromy6.png" width="600" style="float:center">

Computing the fibre over a point via left map requires just solving a linear system of equations.

In [64]:
ForcedSolution=randn(ComplexF64,2)
println("We will force \n",ForcedSolution,"\n to be a solution")
RandomCoefficients=randn(ComplexF64,9)
RandomSystem=evaluate(Sparse.expressions,Sparse.parameters=>RandomCoefficients)
Evaluation=evaluate(RandomSystem,[x,y]=>ForcedSolution)
println("This was not a solution to a random system, it failed by, ")
println(Evaluation)
RandomCoefficients[1]-=Evaluation[1]
RandomCoefficients[5]-=Evaluation[2]

NewRandomSystem=evaluate(Sparse.expressions,Sparse.parameters=>RandomCoefficients)
Evaluation=evaluate(NewRandomSystem,[x,y]=>ForcedSolution)
println("Now the evaluation is ")
println(Evaluation)

We will force 
Complex{Float64}[-0.7651684787097727 + 0.17720003694271172im, -0.5864335104629297 + 0.17918721363361123im]
 to be a solution
This was not a solution to a random system, it failed by, 
Complex{Float64}[-1.1463088198107525 - 1.5446214004277343im, 0.5543662364160301 - 0.7860487602129872im]
Now the evaluation is 
Complex{Float64}[-1.1102230246251565e-16 + 1.1102230246251565e-16im, -8.326672684688674e-17 - 2.7755575615628914e-17im]


In [11]:
mixed_volume(MySparseSystem)

16

Let's apply the idea above until we find 16 points

In [65]:
SolutionsFound=[ForcedSolution]
loop_num=0
while length(SolutionsFound)<16
    loop_num+=1
    γ=[RandomCoefficients,5*randn(ComplexF64,9),3*randn(ComplexF64,9),RandomCoefficients]
    newSolution=solutions(monodromy_loop(Sparse,ForcedSolution,γ))
    if findfirst(x->x[1]≈(newSolution[1][1]) && x[2]≈(newSolution[1][2]),SolutionsFound)==nothing
        push!(SolutionsFound,newSolution[1])
        println("We found a new solution (Total:",length(SolutionsFound),")")
        flush(stdout)
    end
end
println("Finished after ",loop_num, " loops")
    

We found a new solution (Total:2)
We found a new solution (Total:3)
We found a new solution (Total:4)
We found a new solution (Total:5)
We found a new solution (Total:6)
We found a new solution (Total:7)
We found a new solution (Total:8)
We found a new solution (Total:9)
We found a new solution (Total:10)
We found a new solution (Total:11)
We found a new solution (Total:12)
We found a new solution (Total:13)
We found a new solution (Total:14)
We found a new solution (Total:15)
We found a new solution (Total:16)
Finished after 78 loops


<img src="PictureMonodromy3.png" width="500">

Now the offline step is done. 

The online step is just to track these solutions to the system I was interested in!

In [66]:
MySolutions=solutions(solve(Sparse,SolutionsFound;
        start_parameters=RandomCoefficients,target_parameters=MySparseParameters))

16-element Array{Array{Complex{Float64},1},1}:
 [-0.9154322030504802 + 1.7516230804060213e-46im, 0.0 - 0.43878753220451205im]
 [-0.9154322030504802 - 7.703719777548943e-34im, 0.0 + 0.43878753220451205im]
 [-1.7024939578149652 - 2.0662986635548023e-40im, -1.8367099231598242e-40 - 1.5661632403915273im]
 [-1.3730296863493812 + 1.3452465257518244e-43im, -0.6589885172110949 + 1.793662034335766e-43im]
 [0.9154322030504802 + 7.006492321624085e-46im, -5.605193857299268e-45 + 0.43878753220451205im]
 [-1.7024939578149654 + 1.4349296274686127e-41im, 1.7219155529623352e-41 + 1.5661632403915275im]
 [-7.006492321624085e-46 + 1.4130023810945964im, 6.56858655152258e-47 - 0.6615687031225687im]
 [1.7024939578149654 - 5.969307250269429e-40im, 7.346839692639297e-40 + 1.5661632403915275im]
 [1.7024939578149652 + 4.304788882405838e-42im, -1.1479437019748901e-41 - 1.5661632403915273im]
 [0.9154322030504802 - 7.703719777548943e-34im, -1.5407439555097887e-33 - 0.43878753220451205im]
 [1.3730296863493812 + 3.36

Let's check if our solutions are the same as directly solving

In [67]:
sort(MySolutions;lt=(x,y)->abs(real(x[1]))<abs(real(y[1])))

16-element Array{Array{Complex{Float64},1},1}:
 [-7.006492321624085e-46 + 1.4130023810945964im, 6.56858655152258e-47 - 0.6615687031225687im]
 [2.350988701644575e-38 + 1.4130023810945964im, 2.938735877055719e-39 + 0.6615687031225687im]
 [1.925929944387236e-34 - 1.4130023810945962im, 0.0 - 0.6615687031225687im]
 [-1.7256332301709633e-31 - 1.4130023810945964im, 7.703719777548943e-33 + 0.6615687031225687im]
 [-0.9154322030504802 + 1.7516230804060213e-46im, 0.0 - 0.43878753220451205im]
 [-0.9154322030504802 - 7.703719777548943e-34im, 0.0 + 0.43878753220451205im]
 [0.9154322030504802 + 7.006492321624085e-46im, -5.605193857299268e-45 + 0.43878753220451205im]
 [0.9154322030504802 - 7.703719777548943e-34im, -1.5407439555097887e-33 - 0.43878753220451205im]
 [-1.3730296863493812 + 1.3452465257518244e-43im, -0.6589885172110949 + 1.793662034335766e-43im]
 [1.3730296863493812 + 3.363116314379561e-44im, -0.6589885172110949 - 2.2420775429197073e-44im]
 [1.3730296863493812 + 1.1210387714598537e-44im, 0

In [68]:
sort(solutions(solve(MySparseSystem));lt=(x,y)->abs(real(x[1]))<abs(real(y[1])))

16-element Array{Array{Complex{Float64},1},1}:
 [4.70197740328915e-38 - 1.4130023810945962im, 5.877471754111438e-39 - 0.6615687031225687im]
 [-9.4039548065783e-38 + 1.4130023810945962im, -8.816207631167156e-39 + 0.6615687031225687im]
 [-3.009265538105056e-36 + 1.4130023810945962im, 5.64237288394698e-37 - 0.6615687031225687im]
 [6.018531076210112e-36 - 1.4130023810945962im, -5.64237288394698e-37 + 0.6615687031225687im]
 [-0.9154322030504802 + 0.0im, 0.0 + 0.438787532204512im]
 [-0.9154322030504802 - 1.4693679385278594e-39im, -5.877471754111438e-39 - 0.438787532204512im]
 [0.9154322030504802 + 1.4693679385278594e-39im, 5.877471754111438e-39 + 0.43878753220451205im]
 [0.9154322030504802 - 1.8367099231598242e-40im, 7.346839692639297e-40 - 0.438787532204512im]
 [1.3730296863493812 + 3.587324068671532e-43im, 0.6589885172110949 + 3.587324068671532e-43im]
 [-1.3730296863493812 - 1.793662034335766e-43im, -0.6589885172110949 - 7.174648137343064e-43im]
 [-1.3730296863493812 + 1.1754943508222875e-

Maybe you can see how the following schematic is not optimal...

<img src="PictureMonodromy3.png" width="500">

What if we re-used loops?

<img src="PictureMonodromy4.png" width="500">

All of this is implemented in HomotopyContinuation.jl in a smarter way following ideas in Duff et.al.

In [69]:
MonSolve=monodromy_solve(Sparse,ForcedSolution,RandomCoefficients; show_progress=true)

MonodromyResult
• return_code → :heuristic_stop
• 16 solutions
• 128 tracked loops
• random_seed → 0xb60ffe04

In [70]:
MonSolve=monodromy_solve(Sparse; show_progress=true)

MonodromyResult
• return_code → :heuristic_stop
• 16 solutions
• 112 tracked loops
• random_seed → 0x8737886c

Notice, I never told it when to stop. Rather, it did a `heuristic stop'

<bdi style="border:3px; border-style:solid; padding: 0.2em;">
Warning! This may not find all the points you're looking for!
</bdi>

Situation 1: you were unlucky and the loops you took never produced all of the solutions

Situation 2: no matter which loops you took, you'd never produce all the solutions


In [17]:
@var x,y
@var a,b,c
TwoConics = System([(x^2+y^2-1)*((x-0.5)^2+y^2-1),a*x+b*y+c],parameters=[a,b,c])
M=monodromy_solve(TwoConics,[[1,0]],[0,1,0])

MonodromyResult
• return_code → :heuristic_stop
• 2 solutions
• 16 tracked loops
• random_seed → 0x30f6fdf0

So how do you know when you're done in situation 1?

- You knew how many solutions there were to begin with, and can use that as a stopping criterion
- Your solution set looks like $X \cap L$ for some irreducible variety $X$ and a linear space $L$ (see witness sets talk and the trace test)
- You know so much structure about your family that you know exactly which loops you need to take to find all the points
- ...or you don't know and that's something you can live with

## Monodromy Group

The collection of all permutations possible (from loops based at $p \in U$) forms a group $G_\pi$ called the **monodromy group** of $\pi$.

Show $G_\pi$ is a group and that it doesn't depend on the base point $p \in U$.

The following function writes the permutation $g_\gamma$ coming from a loop $\gamma$ as a permutation in single line notation

In [18]:
function monodromy_element(F,BaseSols,BaseParam;γ=nothing)
    #If no loop is given, randomly make one
    if γ==nothing
        γ = [BaseParam] #starting at BaseParam
        for i in 1:3
            push!(γ,2*randn(ComplexF64,length(F.parameters))) #with 3 other parameter values
        end
        push!(γ,BaseParam) #and ending again at BaseParam
    end
    M = solutions(monodromy_loop(F,BaseSols,γ)) #Take the solutions after the loop
    S = (BaseSols) #And those before
    gᵧ= [findfirst(x->x≈S[i],M) for i in 1:length(M)] #and see which solutions were permuted
    return gᵧ
end

monodromy_element (generic function with 1 method)

In [19]:
@var x a b
F = System([x^2 + a * x + b], parameters = [a, b])
#Here's the monodromy element from before written as a permutation
monodromy_element(F,[[-1],[1]],p₀;γ=[p₀,p₁,p₂,p₃,p₀])

2-element Array{Int64,1}:
 2
 1

Let's compute 1000 random monodromy permutations!

In [71]:
#here are 1000 random monodromy elements based at p_0
quadratic_univariate_perms=[monodromy_element(F,[[-1],[1]],Array{ComplexF64,1}(p₀)) for i in 1:1000];

In [72]:
#this is a quick function to tally them
function tally(S)
    D=Dict{Any,Int}()
    for s in S
        D[s]=get(D,s,0)+1
    end
    return D
end

tally (generic function with 1 method)

In [73]:
#here's the tally
tally(quadratic_univariate_perms)

Dict{Any,Int64} with 2 entries:
  [2, 1] => 370
  [1, 2] => 630

Let's investigate the monodromy group of the sparse system from before


<img src="PictureMonodromy9.png" width="500">

In [79]:
sparse_perms=[monodromy_element(Sparse,SolutionsFound,RandomCoefficients) for i in 1:500];
tally(sparse_perms)

Dict{Any,Int64} with 474 entries:
  [8, 9, 10, 4, 3, 5, 14, 2, 1, 6, 11, 12, 13, 7, 16, 15] => 2
  [4, 13, 8, 15, 12, 9, 2, 3, 6, 11, 16, 14, 7, 5, 1, 10] => 1
  [5, 10, 9, 4, 1, 8, 7, 6, 3, 2, 11, 12, 13, 14, 15, 16] => 1
  [3, 6, 11, 16, 8, 12, 5, 13, 4, 9, 15, 7, 14, 2, 10, 1] => 1
  [2, 1, 13, 7, 10, 4, 8, 11, 12, 5, 14, 16, 15, 6, 9, 3] => 1
  [2, 1, 3, 14, 10, 6, 13, 8, 9, 5, 7, 15, 16, 12, 4, 11] => 1
  [9, 8, 10, 15, 6, 5, 11, 2, 1, 3, 16, 14, 7, 4, 12, 13] => 1
  [9, 8, 2, 7, 6, 1, 13, 10, 5, 3, 14, 16, 15, 12, 4, 11] => 1
  [12, 11, 5, 15, 4, 10, 9, 1, 2, 13, 16, 14, 7, 3, 8, 6] => 1
  [1, 2, 3, 4, 5, 6, 15, 8, 9, 10, 11, 12, 13, 16, 7, 14] => 1
  [7, 15, 2, 9, 16, 1, 11, 10, 5, 14, 3, 6, 8, 4, 12, 13] => 2
  [16, 14, 2, 6, 7, 1, 4, 10, 5, 15, 8, 9, 3, 11, 13, 12] => 1
  [5, 10, 3, 4, 1, 6, 14, 8, 9, 2, 11, 12, 13, 7, 16, 15] => 1
  [16, 14, 13, 5, 7, 4, 6, 11, 12, 15, 2, 1, 10, 8, 3, 9] => 1
  [13, 4, 14, 5, 11, 16, 8, 15, 7, 12, 2, 1, 10, 6, 9, 3] => 1
  [12, 11, 3, 16, 4, 

In [82]:
sparse_perms=unique(sparse_perms)
println(length(sparse_perms))
filter!(x->sort(x) != collect([1:16]),sparse_perms)
println(length(sparse_perms))

474
474


#### To compute and describe this monodromy group, we'll need a little help from our favorite group theory software

In [83]:
using GAP; #Warning: may not work with julia 1.6
function describe_group(L)
    L=unique(L)
    Sym=GAP.Globals.SymmetricGroup(length(L[1]))
    GAPL=[]
    for l in L
        push!(GAPL,GAP.Globals.PermList(GAP.julia_to_gap(l)))
    end
    GAPL=GAP.julia_to_gap(GAPL)
    G=GAP.Globals.Subgroup(Sym,GAPL)
    return (GAP.Globals.StructureDescription(G),G)
end

describe_group (generic function with 1 method)

In [84]:
G=describe_group(sparse_perms)[2]

GAP: ((((((((C2 x Q8) : C2) : C2) : C2) : C2) : C3) : C2) : C2) : C2

In [85]:
GAP.Globals.Order(G)

6144

Cn is the cyclic group. x is direct product. : is semidirect product. 

The point here is that it's not the full symmetric group. We'll come back to this example. 

In [86]:
GAP.Globals.IsTransitive(G)

true

### Some facts about monodromy groups

$\bullet$ "Exercise": monodromy_solve can find all solutions if and only if $G_\pi$ is transitive

$\bullet$ Exercise: $G_\pi$ is transitive if and only if $Z$ has a unique irreducible component of top dimension

$\bullet$ $G_\pi$ is $k$-transitive (sends any k-tuple to any other k-tuple (non-ordered)) if and only if the monodromy group of the ''k-th fibre power of the cover $\pi:Z \to Q$'' is transitive



$\bullet$ If $L$ is a generic line in the parameter space, then the monodromy group of $\pi:Z \to Q$ is the same as that of $\pi: Z|_L \to L$. 



<img src="PictureMonodromy11.png" width="500">


$\bullet$ If $\pi:Z \to \mathbb{C}$ is a branched cover, then the discriminant is just points $q_1,\ldots,q_k \in \mathbb{C}$, and loops around these points generate the monodromy group. 


<img src="PictureMonodromy8.png" width="500">

$\bullet$ The monodromy group is a Galois group

# 27 lines on a cubic

**Cayley and Salmon**: There are exactly $27$ lines on every smooth cubic surface in $\mathbb{P}^3$.

Let's solve for these $27$ lines using monodromy, then compute the monodromy group!

*Source: Greg Egan* 

<img src="PictureMonodromy5.gif" width="500">
(The Clebsch cubic)

### Writing the system 

Cubic = $\sum_{i+j+k\leq 3} c_{i,j,k}x^iy^jz^k$

Lines = $t \xrightarrow{\ell_{k}} (t,k_1+k_3t,k_2+k_4t)$

Polynomial system: Cubic($\ell_k$)$=0$ (variables are $k_1,\ldots,k_4$ and parameters are $c_{i,j,k}$)

In [87]:
# lines are parametrized as t->(t,k1+k3t,k2+k4t)
@var k[1:4]
# the coefficient of x^iy^jz^k in a general cubic will be c[i,j,k]
@var c[0:3,0:3,0:3]
# we need these variables too
@var t
@var x,y,z

#here's a cubic
cubic=sum([c[i+1,j+1,k+1]x^i*y^j*z^k for i in 0:3 for j in 0:3-i for k in 0:3-i-j])
#here's our line
line = [t,k[1]+k[3]*t,k[2]+k[4]*t]
#E is cubic(line(t))
E=evaluate(cubic,[x,y,z]=>line)
#and we construct the system where the coefficients of t^i for t=0..3 are zero (i.e. E=0)
F = System(exponents_coefficients(E,[t])[2],parameters=vec(c))

System of length 4
 4 variables: k₁, k₂, k₃, k₄
 64 parameters: c₀₋₀₋₀, c₁₋₀₋₀, c₂₋₀₋₀, c₃₋₀₋₀, c₀₋₁₋₀, c₁₋₁₋₀, c₂₋₁₋₀, c₃₋₁₋₀, c₀₋₂₋₀, c₁₋₂₋₀, c₂₋₂₋₀, c₃₋₂₋₀, c₀₋₃₋₀, c₁₋₃₋₀, c₂₋₃₋₀, c₃₋₃₋₀, c₀₋₀₋₁, c₁₋₀₋₁, c₂₋₀₋₁, c₃₋₀₋₁, c₀₋₁₋₁, c₁₋₁₋₁, c₂₋₁₋₁, c₃₋₁₋₁, c₀₋₂₋₁, c₁₋₂₋₁, c₂₋₂₋₁, c₃₋₂₋₁, c₀₋₃₋₁, c₁₋₃₋₁, c₂₋₃₋₁, c₃₋₃₋₁, c₀₋₀₋₂, c₁₋₀₋₂, c₂₋₀₋₂, c₃₋₀₋₂, c₀₋₁₋₂, c₁₋₁₋₂, c₂₋₁₋₂, c₃₋₁₋₂, c₀₋₂₋₂, c₁₋₂₋₂, c₂₋₂₋₂, c₃₋₂₋₂, c₀₋₃₋₂, c₁₋₃₋₂, c₂₋₃₋₂, c₃₋₃₋₂, c₀₋₀₋₃, c₁₋₀₋₃, c₂₋₀₋₃, c₃₋₀₋₃, c₀₋₁₋₃, c₁₋₁₋₃, c₂₋₁₋₃, c₃₋₁₋₃, c₀₋₂₋₃, c₁₋₂₋₃, c₂₋₂₋₃, c₃₋₂₋₃, c₀₋₃₋₃, c₁₋₃₋₃, c₂₋₃₋₃, c₃₋₃₋₃

 c₃₋₀₋₀ + k₃*c₂₋₁₋₀ + k₃^2*c₁₋₂₋₀ + k₃^3*c₀₋₃₋₀ + k₄*c₂₋₀₋₁ + k₄^2*c₁₋₀₋₂ + k₄^3*c₀₋₀₋₃ + k₄*k₃*c₁₋₁₋₁ + k₄*k₃^2*c₀₋₂₋₁ + k₄^2*k₃*c₀₋₁₋₂
 c₂₋₀₋₀ + k₁*c₂₋₁₋₀ + k₂*c₂₋₀₋₁ + k₃*c₁₋₁₋₀ + k₃^2*c₀₋₂₋₀ + k₄*c₁₋₀₋₁ + k₄^2*c₀₋₀₋₂ + 2*k₃*k₁*c₁₋₂₋₀ + k₃*k₂*c₁₋₁₋₁ + 3*k₃^2*k₁*c₀₋₃₋₀ + k₃^2*k₂*c₀₋₂₋₁ + k₄*k₁*c₁₋₁₋₁ + 2*k₄*k₂*c₁₋₀₋₂ + k₄*k₃*c₀₋₁₋₁ + k₄^2*k₁*c₀₋₁₋₂ + 3*k₄^2*k₂*c₀₋₀₋₃ + 2*k₄*k₃*k₁*c₀₋₂₋₁ + 2*k₄*k₃*k₂*c₀₋₁₋₂
 c₁₋₀₋₀ + k₁

In [88]:
@time MS=monodromy_solve(F; permutations=true, target_solutions_count=27)

  0.040061 seconds (71.41 k allocations: 4.949 MiB)


MonodromyResult
• return_code → :success
• 27 solutions
• 54 tracked loops
• random_seed → 0x598615a1

Monodromy quickly solved the system!

In [89]:
#Output of monodromy_solve knows a fibre pi^(-1)(P) = S, 
# and it knows the permutations corresponding to the loops  used
S=solutions(MS);
P=parameters(MS);
random_elements=[permutations(MS)[:,i] for i in 1:size(permutations(MS),2)]

2-element Array{Array{Int64,1},1}:
 [2, 3, 4, 5, 6, 7, 8, 1, 14, 13  …  21, 16, 15, 20, 12, 11, 9, 27, 26, 25]
 [9, 10, 11, 2, 12, 13, 6, 5, 8, 15  …  25, 14, 24, 26, 18, 27, 23, 22, 4, 19]

In [90]:
#Here we filter out anything funny that happened numerically
filter!(x->!(nothing in x) && sort(x) == collect(1:27),random_elements);
length(random_elements)

2

In [91]:
tally(random_elements)

Dict{Any,Int64} with 2 entries:
  [9, 10, 11, 2, 12, 13, 6, 5, 8, 15  …  25, 14, 24, 26, 18, 27, 23, 22, 4… => 1
  [2, 3, 4, 5, 6, 7, 8, 1, 14, 13  …  21, 16, 15, 20, 12, 11, 9, 27, 26, 2… => 1

In [92]:
Gal=describe_group(random_elements)
println(Gal[1])
GAP.Globals.Order(Gal[2])

GAP: "O(5,3) : C2"


51840

In [95]:
factorial(big(27))

10888869450418352160768000000

# Take away number 1: 
Every situation in numerical algebraic geometry in which you can do homotopy continuation, you can compute a monodromy group.

# Take away number 2:
If it is smaller than the full symmetric group, PAY ATTENTION! It means there is structure there.

# Take away number 3:
The monodromy group is as big as it could be...

The monodromy group of the 27 lines is exactly the group of permutations which are possible given how the $27$ lines must intersect.

**Schlafli graph: 27 vertices <-> lines, edges <-> lines do NOT intersect**
<img src="PictureMonodromy6.svg" width="500">
Source: By Claudio Rocchini - Own work, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=11045032

## The monodromy is as big as it could be (again)
<img src="PictureMonodromy9.png" width="500">

Solving $$f(x^2,y^2) = g(x^2,y^2)=0$$ is the same as solving $$f(a,y^2)=g(a,y^2)=0$$ and then taking $x=\pm \sqrt{a}$ which is the same as just solving $$f(a,b)=g(a,b)=0$$ and then taking $y=\pm \sqrt{b}$.

This is an example of the branched cover factoring through smaller branched covers...

<img src="PictureMonodromy7.png" width="800">

As the parameters of the coefficients move, this relationship on the respective fibres of the cover 
$$Z \xrightarrow{2\text{-to-}1} Z_1 \xrightarrow{2\text{-to-}1} Z_2 \xrightarrow{4\text{-to-}1} \mathbb{C}^4 \times \mathbb{C}^5$$ needs to be respected.

So the monodromy group can at most be the automorphism group of $4$ copies of the binary tree above. 

Taking the other decomposition order will force it to be even smaller!

Groups that respect this kind of structure are called **imprimitive groups**.

These are permutation groups that preserve partitions.
i.e. $(a_1,b_1) -> (a_2,b_2)$ in the above example implies that $\{(\pm a_1, \pm b_1)\} \to \{(\pm a_2, \pm b_2)\}$ so in particular, the partition $$\{(\pm a_1, \pm b_1)\} \sqcup \{(\pm a_2, \pm b_2)\} \sqcup \{(\pm a_3, \pm b_3)\} \sqcup \{(\pm a_4, \pm b_4)\}$$ is preserved.


In [96]:
#Remember G was the monodromy group of this sparse system
GAP.Globals.IsTransitive(G)

true

In [97]:
GAP.Globals.IsPrimitive(G)

false

In [99]:
GAP.Globals.Blocks(G,GAP.julia_to_gap([i for i in 1:16]))

GAP: [ [ 1, 5 ], [ 2, 10 ], [ 3, 8 ], [ 4, 12 ], [ 6, 9 ], [ 7, 16 ], [ 11, 13 ], [ 14, 15 ] ]



$\bullet$ The monodromy group is imprimitive $\iff$ there is a nontrivial factoring of the branched cover like this.



The symmetry is rarely as obvious as above. 


This kind of structure can be taken advantage of when it comes to solving via monodromy.

[[**Solving Parameterized Polynomial Systems with Decomposable Projections**](https://arxiv.org/abs/1612.08807)] Carlos Améndola, Julia Lindberg, Jose Israel Rodriguez

[[**Solving Decomposable Sparse Systems**](https://arxiv.org/abs/2001.04228)] Taylor Brysiewicz, Jose Israel Rodriguez, Frank Sottile, Thomas Yahl)

If you're interested about monodromy groups of sparse polynomial systems, check out 

[[**Galois theory for general systems of polynomial equations**](https://arxiv.org/abs/1801.08260)] Alexander Esterov.